# $\rho'$ analysis notes

## GIRD Selection criteria:

These criteria were applied on the selection stage(GRID):

Events:

    * >= 4 tracks
    
Tracks:

    * Has Point On inner or outer ITS Layer
    * Not ITS SA
    * |dca1| < 3 && |dca0| < 3;

### Data info

In [2]:
from modules import events, dfs, np, pd, hep, plt, Pi0, PiPlus, Rho0
from modules.data.info.DataInfo import DataInfo

DataInfo()


name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
RunNum               | int32_t                  | AsDtype('>i4')
PeriodNumber         | uint32_t                 | AsDtype('>u4')
OrbitNumber          | uint32_t                 | AsDtype('>u4')
BunchCrossNumber     | uint16_t                 | AsDtype('>u2')
Mass                 | float                    | AsDtype('>f4')
Pt                   | float                    | AsDtype('>f4')
Q                    | int16_t                  | AsDtype('>i2')
Rapidity             | float                    | AsDtype('>f4')
Phi                  | float                    | AsDtype('>f4')
ZNAenergy            | float                    | AsDtype('>f4')
ZNCenergy            | float                    | AsDtype('>f4')
ZPAenergy            | float                    | AsDtype('>f4')
ZPCenergy            | float                    | AsDtype(

## Analysis criteria

Let's take from input date only events with 4 tracks and zero total charge.
Also prepare mask for tracks that satisfy standatd criteria contains further conditions for the TPC:

* |NumberOfSigmaTPCPion| < 3
* Number of TPC Clusters > 50
* TPCRefit


In [3]:
from modules.data.selection import dfs4Tracks, TPCMask, GetITSnTPCTracksDF, dfs4TracksLowPt, TPCMaskLowPt

## Low energy tracks and TPC

There is an idea about tracks with small energies (low pt) not able to reach TPC.
We can try to estimate influence of TPC criteria to the statistics.

Let's see what happend in case of we will plot pt for any combination of TPC tracks:

### Event with std criteria

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|+|

### Let's start to take also such combinations to the sample:

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|-|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|-|-|-|-|

We can see statistics gain ration equal 2.5 between std criteria and total combinations:


In [4]:
from matplotlib.patches import Rectangle
from modules.physics import kinematics

%matplotlib widget


plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

fig.suptitle(f'4track events pt', fontsize=32)
tpcimpPt = []
colors = ['red','green', 'yellow', 'orange', 'black']
labels = ['ITS & (>= 0TPC)','ITS & (>= 1TPC)', 'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & ( =  4TPC)']

for i in range(5):
    ITSnTPCEvents = pd.unique(dfs4Tracks.reset_index().entry)[TPCMask.groupby('entry').sum() >= i]
    ITSnTPCTracks = dfs4Tracks.loc[ITSnTPCEvents]
    tmpPt = kinematics(ITSnTPCTracks).EventsVectors.pt
    tpcimpPt.append(tmpPt)
    ax.hist(tpcimpPt[i], bins=100, range=(0,2), histtype='step', color=colors[i], linewidth=2, label=labels[i]+f';{(tmpPt<0.15).sum()}')

ax.set_xlabel('$p_t, GeV$')
ax.set_ylabel('# events')

ax.add_patch(Rectangle((0,0.15),0.15,900,fc='lightgrey',alpha=0.4))

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now  let's try to see what tracks we lost from signal area and what contribution they have:

In [5]:
from modules.physics.analysis.ITSvsTPC import *

%matplotlib widget

ShowComparison('$p_t$',[AllTPCTracksPt, ITSDiffTPCTracksPt, TotalLowPt], 'GeV', ['ITS&&TPC', 'OnlyITSFromNTPCEvents', 'ITS||TPC'])




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TPC and ITS has different coverage for polar angle:

![img1](https://camo.githubusercontent.com/9a7ab40d0f74a866a7095986644134a0f72cc58b/68747470733a2f2f70702e757365726170692e636f6d2f633835323231362f763835323231363738332f3131396137332f304f76685f6c544b4e7a552e6a7067)

Perhaps we have tracks that not only can't reach TPC, but also has $\theta$ values that TPC doesn't cover.

Below we can see polar angle distribution for tracks that covers three cases:

1. All tracks from events were reconstructed by ITS and TPC
2. Only ITS tracks from events with only part TPC tracks. Here tracks that not reconstructed by TPC
3. All tracks from events were reconstructed by ITS or TPC

We can see small gaps with for the second case, that allow to speak about correctness of the suggestion, but anyway low energy of tracks is the main reason why TPC can't reconstructed tracks.  

In [6]:
ShowComparison('$\\theta$',[AllTPCTracksTheta, ITSDiffTPCTracksTheta, TotalLowPtTheta], '$^\\circ$', ['ITS&&TPC', 'OnlyITSFromNTPCEvents', 'ITS||TPC'], nBins=80, ranges=(25,150))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mass
Let's see on the mass distribution of the events


In [7]:
from modules.physics.analysis.mass import GetEventMass

%matplotlib widget

plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

fig.suptitle(f'4track events Mass', fontsize=32)
tpcimpMass = []
colors = ['yellow','green', 'red', 'orange', 'black']
labels = ['ITS & (>= 0TPC)','ITS & (>= 1TPC)', 'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & ( =  4TPC)']

for i in range(2,5):
    tmpMass = GetEventMass(i)
    tpcimpMass.append(tmpMass)
    ax.hist(tpcimpMass[-1], bins=100, range=(0,4), histtype='step', color=colors[i], linewidth=2, label=labels[i])

ax.set_xlabel('$Mass, GeV$')
ax.set_ylabel('# events')

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from modules.physics.analysis.mass import GetEventMass

%matplotlib widget

plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

fig.suptitle(f'4track events Mass', fontsize=32)
tpcimpMass = []
colors = ['yellow','green', 'red', 'orange', 'black']
labels = ['ITS & (>= 0TPC)','ITS & (>= 1TPC)', 'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & ( =  4TPC)']
b = 100
r = 0.5,3.5
for i in range(2,3):
    tmpMass = GetEventMass(i)
    tpcimpMass.append(tmpMass)
    counts,bin_edges = np.histogram(tpcimpMass[-1], bins=b, range=r)
    bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
    errs = np.sqrt((counts - np.average(counts))**2/len(counts))
    ax.errorbar(bin_centres, counts, yerr=errs, fmt='.', label=labels[i], color=colors[i])

val=(r[1]-r[0])*1000 // b
ax.set_xlabel('$Mass, GeV$')
ax.set_ylabel(f'events / {val}MeV')

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Pions subsystems

In our process 4 pions were producted. The most probably intermediate state including two pions and $\rho$ i.e.
$$\rho' \rightarrow \rho \ \pi^+ \pi^- \rightarrow \pi^+ \pi^- \pi^+ \pi^-$$ 

We can see this on distribution of mass that can be obtained as all combinations of pairs from intial four tracks, i.e. only four pairs:

![img](https://camo.githubusercontent.com/53a52e2a6d4ae7112d74f4073c979a51166170d8/68747470733a2f2f70702e757365726170692e636f6d2f633835333632342f763835333632343436372f34383466332f5431375a754b597062526f2e6a7067)

Here we can plot two distirbutions:
1. Make all possible(4) combinations of pairs. Then take lightest and pair that belong to one combination with that. Plot masses of these two pairs.
2. Plot masses of masses from possible combinations.

In [9]:
from modules.physics.analysis.mass import LiteHeavyRecoil, LiteHeavyTotal, ShowMassComaprison

%matplotlib widget
ShowMassComaprison(LiteHeavyRecoil, 'Lightest and Recoil Pairs')
ShowMassComaprison(LiteHeavyTotal, 'All Pairs')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


As we can see above, for second case (all possible pairs) we've got stronger signal in comparison with light-recoil pair as it made in [STAR work](http://arxiv.org/abs/0912.0604v2). Let's build 2d distirbuition and marginals component separately:

In [10]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np



plt.style.use(hep.style.ROOT)

fig = plt.figure(figsize=(15, 7))

gs = GridSpec(4,4,wspace=0.1,hspace=0.1)

ax_joint = fig.add_subplot(gs[1:4,0:3])
ax_marg_x = fig.add_subplot(gs[0,0:3])
ax_marg_y = fig.add_subplot(gs[1:4,3])

ax_joint.hist2d(LiteHeavyTotal.Recoil, LiteHeavyTotal.Lite, bins=(50, 50), range=[(0, 2), (0, 2)], cmap=plt.cm.jet)
_ = ax_marg_y.hist(LiteHeavyTotal.Lite, bins=100, range=(0, 2), histtype='step', color='blue', linewidth=2, label='lite pair',orientation="horizontal")
_ = ax_marg_x.hist(LiteHeavyTotal.Recoil, bins=100, range=(0, 2), histtype='step', color='red', linewidth=2, label='rest pair')


ax_joint.set_ylabel('Lightest pair Mass, GeV')
ax_joint.set_xlabel('Recoiling pair Mass, GeV')

# ax_marg_y.set_xlabel('$Mass, GeV$')
ax_marg_y.set_xlabel('# events')
ax_marg_x.yaxis.set_label_position("right")
ax_marg_x.xaxis.set_ticks([])
ax_marg_x.xaxis.set_ticks_position('none')
ax_marg_y.yaxis.set_ticks_position('none')
ax_marg_y.yaxis.set_ticks([])

# ax_marg_x.set_xlabel('$Mass, GeV$')
ax_marg_x.set_ylabel('# events')
ax_marg_y.xaxis.set_label_position("top")
ax_marg_x
ax_joint.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [11]:
%matplotlib widget


plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

counts1, bins1  = np.histogram(LiteHeavyRecoil.Recoil, bins=100, range=(0,2))
counts2, bins2  = np.histogram(LiteHeavyTotal.Recoil, bins=100, range=(0,2))

_ = ax.hist(counts1/counts2,bins=bins1, histtype='step', color='black', linewidth=2)


plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

counts1, bins1  = np.histogram(LiteHeavyRecoil.Lite, bins=100, range=(0,2))
counts2, bins2  = np.histogram(LiteHeavyTotal.Lite, bins=100, range=(0,2))

_ = ax.hist(counts1/counts2,bins=bins1, histtype='step', color='black', linewidth=2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## $\eta$ distribution for pairs

In [12]:
from modules.physics.analysis.mass import LiteHeavyRecoil, LiteHeavyTotal

## Cross section

Let's see to cross section of my events.

For this we should take luminosity of runs.

> Unfortunately file that I have to use for getting luminosity have a reference to special class AliTriggerInfo and moreover it packed into TObjArray, so I can't read it via uproot4. This is the reason why I used pure root again. [Here is the script](https://github.com/bdrum/cern-physics/blob/master/root-cpp/RhoPrime/macro/lumi.C) that I used.

Then let's see how much events do we have in each run.

$$L = \sigma \frac{\delta N}{\delta t}$$, this means that 

$$\sigma \approx \frac{N}{L}$$

Cross section of phenomena should be flat and independent from runs. 

Let's check it:

In [14]:
from modules.physics.analysis.crossection import *

ShowCrossSection()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …